In [1]:
import json

import PyPDF2 as pdf
import re

In [18]:
pdf_path = '/home/hubert/Desktop/sds/64-17-5.pdf'

reader = pdf.PdfReader(pdf_path)
print(reader.pages[0].extract_text())

 
 
Millipore - 1.08543  Strona 1  z  23 
 
The life science business of Merck operates as MilliporeSigma in 
the US and Canada  
  
 
 
  
 
 
KARTA CHARAKTERYSTYKI  
zgodnie z Rozporz ądzeniem WE 1907/2006  Wersja  8.10 
Aktualizacja  27.03.2023  
Wydrukowano dnia  18.11.2023   
SEKCJA 1: Identyfikacja substancji/mieszaniny i identyfikacja przedsi ębiorstwa  
1.1 Identyfikatory produktu  
Nazwa wyrobu  : Etanol dla biologii molekularnej  
 
Numer produktu  : 1.08543  
Numer katalogowy  : 108543  
Marka  : Millipore  
Numer indeksowy  : 603-002-00-5 
Nr REACH  : 01-2119457610 -43-XXXX  
Nr CAS  : 64-17-5 
 
1.2 Istotne zidentyfikowane zastosowania substancji lub mieszaniny oraz 
zastosowania odradzane  
Zastosowania 
zidentyfikowane  : Badania/analiza biochemiczna, Odczynnik do analizy  
1.3 Dane dotycz ące dostawcy karty charakterystyki  
Firma  : Merck Life Science Sp.z.o.o.   
Szelągowska 30  
PL-61-626 POZNAN   
Numer telefonu  : +48 61 8290 -100 
Faks : +48 61 8290 -120 
Adres e 

In [23]:
class ParserPL():
    def __init__(self, path):
        self.reader = pdf.PdfReader(path)
        self.name = re.compile(r'(?<=(Nazwa wyrobu  :)).*')
        self.cas = re.compile(r'(?<=(Nr CAS  :)).*')
        self.sigma = re.compile(r'sigma', re.IGNORECASE)
        with open('data/h_dict.json', 'r') as file:
            self.h_dict = json.load(file)
    
    def verify_sigma(self):
        for page in self.reader.pages:
            text = page.extract_text()
            if re.findall(self.sigma, text):
                return True
        return False
    
    def return_name(self):
        for page in self.reader.pages:
            text = page.extract_text()
            name = re.findall(self.name, text)
            if name:
                return name[0]
    
    def return_cas(self):
        for page in self.reader.pages:
            text = page.extract_text()
            cas = re.findall(self.cas, text)
            if cas:
                return cas[0]
    
    def return_EUH_description(self):
        EUH_description = []
        EUHs = self.search_for_EUH()
        for EUH in EUHs:
            EUH_description.append(f'{EUH} - {self.h_dict[EUH]}')
        return ('. ').join(EUH_description)
        
    def search_for_EUH(self):
        for page in self.reader.pages:
            text = page.extract_text()
            if re.findall(re.compile(r'SEKCJA 16: Inne informacje'), text):
                h_matches = re.findall(re.compile(r'H\d[0-9]*'), text)
                euh_matches = re.findall(re.compile(r'EUH\d[0-9]*'), text)
                all_matches = h_matches + euh_matches
                return all_matches

In [26]:
parser = ParserPL('data/example_sigma_PL.pdf')
print(parser.verify_sigma())

True
